In [1]:
# Imports
import openai
import os
import requests
from utils import check_affordance_func

# Notebook Imports
from IPython.display import Image
from IPython.display import display
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import random
from utils import generate_image
from utils import get_completion
from pathlib import Path
from segment import GibsonSAM
from config import CLASSES
from dataclasses import dataclass


In [ ]:
@dataclass
class ModelConfig:
    HOME: str = "/home/aregbs/Desktop/gibson-afford"
    GROUNDING_DINO_CONFIG_PATH: str = os.path.join(HOME, "GroundingDINO/groundingdino/config/GroundingDINO_SwinT_OGC.py")
    GROUNDING_DINO_CHECKPOINT_PATH: str = os.path.join(HOME, "weights", "groundingdino_swint_ogc.pth")
    SAM_CHECKPOINT_PATH: str = os.path.join(HOME, "weights", "sam_vit_h_4b8939.pth")
    SAM_ENCODER_VERSION: str = "vit_h"
    

In [2]:
os.environ['OPENAI_API_KEY'] = ""
openai.api_key = os.getenv("OPENAI_API_KEY")

In [3]:
object = "pot", "plate", "apple", "tomatoes", "knife", "bowl", "mug","onion"
object_scene = "kitchen"

In [16]:
prompt = f"""You are a prompt generator generate 10 prompts based strictly on the template"""
# Define your template
template = """
        a {object_scene} with {object}, {object} and an {object}, {object},
        a {object}, a {object} and an {object} on a {object} {object_scene},
        an {object} on a {object}, a {object}  and a {object} separate, {object_scene}
        
        object = "pot", "plate", "apple", "tomatoe", "knife", "bowl", "2
        object_scene = "kitchen"
        """

In [ ]:
"egg", "bowl", "pot", "plate", "mug", "glass cup", "knife","apple","onion","tomatoe","fork", "spoon"

In [17]:
template

'\n        a {object_scene} with {object}, {object} and an {object}, {object},\n        a {object}, a {object} and an {object} on a {object} {object_scene},\n        an {object} on a {object}, a {object}  and a {object} separate, {object_scene}\n        \n        object = "pencil", "laptop", "book", "scissors", "water bottle", "mug"\n        object_scene = "office"\n        '

In [18]:
prompts = get_completion(template=template, prompt=prompt)

In [19]:
prompts

[{'label': 1,
  'prompt': 'A cluttered office desk with a pencil, laptop, book, scissors, and a water bottle.'},
 {'label': 2,
  'prompt': 'A sleek laptop, a worn book, and a colorful mug on a wooden desk in an office.'},
 {'label': 3,
  'prompt': 'An open book, a sharp pair of scissors, and a half-empty water bottle on a glass desk in an office.'},
 {'label': 4,
  'prompt': 'A pencil, a laptop, and a mug on a messy desk in an office.'},
 {'label': 5,
  'prompt': 'A closed book, a shiny pair of scissors, and a full water bottle on a metal desk in an office.'},
 {'label': 6,
  'prompt': 'A laptop, a pencil, and a mug on a wooden desk in an office.'},
 {'label': 7,
  'prompt': 'A water bottle on a glass desk, a book on a wooden shelf, and a pair of scissors on a metal filing cabinet in an office.'},
 {'label': 8,
  'prompt': 'A pencil, a laptop, and a book on a cluttered desk in an office.'},
 {'label': 9,
  'prompt': 'A mug, a water bottle, and a pair of scissors on a wooden desk in an 

In [ ]:
output_folder = "data"
base_images_number = 2
variation_per_image = 1
# Check if output directory for noisey files exists and create it if it doesn't
DATA_DIR = Path.cwd() / output_folder

DATA_DIR.mkdir(exist_ok=True)

In [ ]:
generate_image(base_images_number=base_images_number, labels=prompts, output_folder=DATA_DIR, variation_per_image=variation_per_image)

In [ ]:
image_path= "/home/aregbs/Desktop/gibson-afford/gen_data/out_door/1.HIl3wG8RIwImzWXhe3hKFMIGKPc%3D.png"
output_folder = "mask_outdoor"
DATA_DIR = Path.cwd() / output_folder

DATA_DIR.mkdir(exist_ok=True)

gibson = GibsonSAM(image_path, CLASSES, model_config= ModelConfig())
labels = gibson.get_image_annotations(DATA_DIR)


In [ ]:
labels

In [ ]:
objects_string = ", ".join(labels)

In [ ]:
objects_string

In [ ]:
labels = ['apple', 'banana', 'rubber plate', 'mug']

In [ ]:
task = "Generate prompts based on the affordance in this format: {action} the {objects} "

In [ ]:
affordance = check_affordance_func(objects=labels, query_template= query_template, task=task)

In [ ]:
affordances = [item.strip()[3:] for item in affordance.split('\n') if item.strip() != '']

In [ ]:
affordances

In [ ]:
def process_dalle_images(response, filename, image_dir):
    # save the images
    urls = [datum["url"] for datum in response["data"]]  # extract URLs
    images = [requests.get(url).content for url in urls]  # download images
    image_names = [f"{filename}_{i + 1}.png" for i in range(len(images))]  # create names
    filepaths = [os.path.join(image_dir, name) for name in image_names]  # create filepaths
    for image, filepath in zip(images, filepaths):  # loop through the variations
        with open(filepath, "wb") as image_file:  # open the file
            image_file.write(image)  # write the image to the file

    return filepaths

In [ ]:
from pathlib import PosixPath

paths = Path("/home/aregbs/Desktop/gibson-afford/gen_data/mask_outdoor")
paths = list(paths.iterdir())
mapped_data = []
for affordance in affordances:
    split_affordance = affordance.split(' ')
    object_name = split_affordance[2]
    matching_paths = [str(path) for path in paths if path.stem == object_name]
    for path in matching_paths:
        edit = {'prompt': affordance, 'mask_path': path}
        mapped_data.append(edit)

print(mapped_data)


In [ ]:
for i, edit in enumerate(mapped_data):
    prompt = edit["prompt"]
    mask_path = edit["mask_path"]

    edit_response = openai.Image.create_edit(
        image=open(image_path, "rb"),
       mask=open(mask_path, "rb"), 
        prompt=prompt,
        size="256x256",
        response_format="url"
    )

    edit_filepaths = process_dalle_images(edit_response, f"Image_{prompt}", DATA_DIR)

In [ ]:
"/home/aregbs/Desktop/gibson-afford/gen_data/"[:]

In [ ]:
image_path[-1]

In [ ]:
[-1] == "/"